<a href="https://colab.research.google.com/github/Tensai-Pro/SRI-PracticeTasks/blob/master/BERTopic%20for%20chats/BERTopic%20for%20chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install bertopic

In [6]:
# !pip install transformers

In [33]:
# !pip install datasets

In [35]:
# !pip install --upgrade accelerate

In [2]:
import pandas as pd
import numpy as np

# Подготовка данных

In [3]:
data = pd.read_csv('/content/data_web.csv', names=['date', 'from', 'text'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    689 non-null    object
 1   from    689 non-null    object
 2   text    689 non-null    object
dtypes: object(3)
memory usage: 16.3+ KB


In [4]:
docs = data['text'].tolist()
print(type(docs))
print(docs[:3])

<class 'list'>
['Настройка https на localhost . ', 'Новый сервис капчи от яндекса на замену сервису ReCAPTCHA от Гугла. Кстати прикольно, не думал, что капча это вариация на тему теста Тьюринга. Если не знаете что это за тест, рекомендую погуглить, развивает кругозор,   . ', 'Введение в svg анимацию доя верстальщиков  . ']


# Подготовка модели BERTopic

In [5]:
from transformers.pipelines import pipeline

In [6]:
# чистая модель rubert_tiny
embedding_model = pipeline('feature-extraction', model='cointegrated/rubert-tiny2')

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
import nltk;
nltk.download('stopwords');
from nltk.corpus import stopwords;
stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

In [10]:
from bertopic import BERTopic

In [11]:
topic_model = BERTopic(language="multilingual", embedding_model=embedding_model,
                       vectorizer_model=vectorizer_model,
                       calculate_probabilities=True, verbose=True)

topics, probs = topic_model.fit_transform(docs)

100%|██████████| 689/689 [00:11<00:00, 58.67it/s]
2023-05-12 15:53:14,735 - BERTopic - Transformed documents to Embeddings
2023-05-12 15:53:36,391 - BERTopic - Reduced dimensionality
2023-05-12 15:53:36,474 - BERTopic - Clustered reduced embeddings


# Извлечение тем

In [12]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,0,664,0_это_коллеги_очень_спасибо
1,1,25,1_позиция абсолютная_позиция_группы позиция_са...


In [13]:
topic_model.get_topic(0)

[('это', 0.029800860487000688),
 ('коллеги', 0.014868404811310723),
 ('очень', 0.013435492129944578),
 ('спасибо', 0.012331723320136684),
 ('js', 0.009640960766043613),
 ('сколько', 0.009241183131225064),
 ('всем', 0.008836974315738884),
 ('который', 0.008428086315968074),
 ('google', 0.008014241902650541),
 ('проект', 0.008014241902650541)]

In [14]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [15]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,0,664,0_не_на_что_как
1,1,25,1_позиция абсолютная_позиция_группы позиция_са...


# Переобучение rubert_tiny

## Подготовка данных из huggingface hub

In [16]:
from datasets import load_dataset

In [36]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [17]:
dataset = load_dataset('YuRayne/sri_pracrice_2')

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# dataset['train']

In [19]:
len(dataset['train']['date'])

689

In [20]:
dataset['train'][10]

{'date': '2022-07-04 09:30:50',
 'from': 'Антон Евтюшкин',
 'text': 'Статья обзор сервиса Github Copilot, искусственного интеллекта, который помогает вам писать код.   . '}

## Переобучение модели

In [21]:
from transformers import (AutoModel, AutoModelForMaskedLM,
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [27]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModelForMaskedLM.from_pretrained('cointegrated/rubert-tiny2')

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
# разбивает датасет на батчи
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# берет каждую строчку текстового файла
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='/content/examples_web.txt',
    block_size=128,
)

In [29]:
trained_path = 'train/rubert-tiny2_trained'

training_args = TrainingArguments(
    output_dir=trained_path,
    overwrite_output_dir=True,
    num_train_epochs=2,
    save_steps=5000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [31]:
saved_path = 'trained_path/'
trainer.train()
trainer.save_model(saved_path)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


## Как работает AutoTokenizer и AutoModel

In [35]:
import torch

In [40]:
def tokenize(text, tokenizer):
  print()
  # делит на слова (токены)
  print(tokenizer.tokenize(text))

  t = tokenizer(text[0], text[1], padding=True, truncation=True)
  # делит на числа
  print('t:', t)

  encoded_seq = t['input_ids']
  print('encoded_seq:', encoded_seq)

  decoded_seq = tokenizer.decode(encoded_seq)
  print('decoded_seq:', decoded_seq)

tokenize(['привет мир.', 'Как дела?'], tokenizer)


['привет', 'мир', '.', 'Как', 'дела', '?']
t: {'input_ids': [2, 32840, 9311, 18, 3, 7063, 7843, 35, 3], 'token_type_ids': [0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
encoded_seq: [2, 32840, 9311, 18, 3, 7063, 7843, 35, 3]
decoded_seq: [CLS] привет мир. [SEP] Как дела? [SEP]


In [36]:
def tokenize_batch(text, tokenizer):
  print()

  encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
  print('encoded_inputs:')
  print(encoded_inputs)

tokenize_batch(['привет.', 'Как у тебя дела?', 'утка ходит'], tokenizer)


encoded_inputs:
{'input_ids': tensor([[    2, 32840,    18,     3,     0,     0,     0],
        [    2,  7063,   331, 29852,  7843,    35,     3],
        [    2, 78415,   872, 38665,     3,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0]])}


Функция из документации к rubert_tiny

In [38]:
model_original = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
def embed_bert_cls(text, model, tokenizer):
  t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      model_output = model(**{k: v.to(model.device) for k, v in t.items()})

  print(model_output.last_hidden_state.shape)

  embeddings = model_output.last_hidden_state[:, 0, :]
  pooler = model_output.pooler_output
  # print(embeddings.shape)
  # print(pooler.shape)

  # print(embeddings)
  embeddings = torch.nn.functional.normalize(embeddings)
  return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model_original, tokenizer)[:5])

torch.Size([1, 4, 312])
[ 0.01934799 -0.02039804 -0.00845532 -0.06858292 -0.00167201]


# BERTopic с дообученной моделью векторного представления

## Извлечение векторных представлений

In [42]:
model_custom = AutoModel.from_pretrained('/content/trained_path', output_hidden_states=False)

Some weights of the model checkpoint at /content/trained_path were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /content/trained_path and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']

In [44]:
docs[3]

'Хороший интерактивный мануал по использованию Git  . '

дообучить токенайзер?

In [47]:
def embed_bert_cls_custom(text, model, tokenizer):
  t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      model_output = model(**{k: v.to(model.device) for k, v in t.items()})

  embeddings = model_output.last_hidden_state[:, 0, :]
  embeddings = torch.nn.functional.normalize(embeddings)
  return embeddings[0].cpu().numpy()

In [48]:
print(embed_bert_cls_custom(docs[0], model_custom, tokenizer).shape)

(312,)


In [46]:
docs_embeddings = [embed_bert_cls_custom(doc, model_custom, tokenizer) for doc in docs]

In [52]:
print(type(docs_embeddings))
print(len(docs_embeddings))
print(docs_embeddings[0].shape)

<class 'list'>
689
(312,)


In [54]:
docs_embeddings = np.array(docs_embeddings)
print(type(docs_embeddings))
print(docs_embeddings.shape)

<class 'numpy.ndarray'>
(689, 312)


## Извлечение тем

In [55]:
bertopic_model = BERTopic(language="multilingual")
topics, probs = bertopic_model.fit_transform(docs, docs_embeddings)

2023-05-12 17:07:31,085 - BERTopic - Reduced dimensionality
2023-05-12 17:07:31,136 - BERTopic - Clustered reduced embeddings


In [56]:
freq2 = bertopic_model.get_topic_info(); freq2.head(10)

,Topic,Count,Name
0,0,660,0_не_на_что_как
1,1,29,1_равно_если_аршинов_сайтописец
